In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import datetime as dt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go
import datetime

from sklearn.compose import make_column_transformer

# Scaling
from sklearn.preprocessing import scale 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler

# Modelling
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Importing plotly and cufflinks in offline mode
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import plotly.graph_objects as go

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

# Figure&Display options
plt.rcParams["figure.figsize"] = (16, 9)
pd.set_option('max_colwidth',200)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# !pip install termcolor
import colorama
from colorama import Fore, Style  # makes strings colored
from termcolor import colored
from termcolor import cprint

import ipywidgets
from ipywidgets import interact


# !pip install squarify
import squarify as sq

In [ ]:
df=pd.read_excel(r'C:\\Users\\HP\\Downloads\\sales_data.xlsx')
df.head(500)

In [ ]:
data=df.drop("CustomerID",axis=1)

In [ ]:
data.info()

df.isnull().sum()

In [ ]:
data.describe()

# In data analysis What all things we do
1.Missing Vlues
2.Explore About the numerical Variables
3.fiding relaonship betwwen feaures

In [ ]:
data.isnull().sum()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.set(rc = {'figure.figsize':(20,10)})
sns.barplot(data=data ,x="TOTAL_ORDERS",y="REVENUE")


In [ ]:
sns.barplot(data=data ,x="TOTAL_ORDERS",y="AVERAGESHIPPING")

In [ ]:
sns.barplot(data=data ,x="TOTAL_ORDERS",y="CARRIAGE_REVENUE")

In [ ]:
sns.barplot(data=data ,x="WEEK1_DAY01_DAY07_ORDERS",y="WEEK1_DAY01_DAY07_REVENUE")

In [ ]:
sns.barplot(data=data ,x="WEEK4_DAY24_DAY31_ORDERS",y="WEEK4_DAY24_DAY31_REVENUE")

In [ ]:
import datetime as dt
from datetime import datetime
from datetime import timedelta

In [ ]:
min_FIRST_ORDER_DATE = min(df['FIRST_ORDER_DATE'])
min_FIRST_ORDER_DATE

In [ ]:
max_FIRST_ORDER_DATE = max(df['FIRST_ORDER_DATE'])
max_FIRST_ORDER_DATE

In [ ]:
df['last_purchase_date'] = df.groupby('CustomerID')['FIRST_ORDER_DATE'].transform(max)

In [ ]:
df['last_purchase_date'] = pd.to_datetime(df['last_purchase_date']).dt.date

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['last_purchase_date'] = pd.to_datetime(df['last_purchase_date'])
df['FIRST_ORDER_DATE'] = pd.to_datetime(df['FIRST_ORDER_DATE'])

In [ ]:
df['ref_date'] = df['FIRST_ORDER_DATE'].max() + timedelta(days=7)

In [ ]:
df['ref_date'] = df['ref_date'].dt.date

In [ ]:
df['ref_date'] = pd.to_datetime(df['ref_date'])

In [ ]:
df.info()

2. Create a new column called Date which contains the invoice date without the timestamp

In [ ]:
df['date'] = pd.to_datetime(df['FIRST_ORDER_DATE'])

In [ ]:
df['date'] = df['date'].dt.date

In [ ]:
df.sample(4)

3. Group by CustomerID and check the last date of purchase

In [ ]:
customer_recency = pd.DataFrame(df.groupby('CustomerID', as_index=False).date.max())
customer_recency.head()

In [ ]:
df[df["CustomerID"] == 5][["CustomerID", 'last_purchase_date']]

In [ ]:
customer_recency.info()

4. Calculate the days since last purchase

In [ ]:
df["customer_recency"] = df["ref_date"] - df["last_purchase_date"]
df[["CustomerID", 'last_purchase_date', "ref_date", "customer_recency"]]

In [ ]:
df['recency2'] = pd.to_numeric(df['customer_recency'].dt.days.astype('int64'))
df[["CustomerID", 'last_purchase_date', "ref_date", "customer_recency", 'recency2']]

In [ ]:
customer_recency = df.groupby('CustomerID', as_index=False)['recency2'].mean()
customer_recency.rename(columns={'recency2':'Recency'}, inplace=True)
customer_recency.sort_values(by='Recency', ascending=False).head()

5. Drop Last_Purchase_Date since we don't need it anymore

In [ ]:
df.drop(['last_purchase_date'], axis = 1, inplace=True)
df.head(3)

6. Plot RFM distributions

In [ ]:
%matplotlib inline

plt.figure(figsize=(10, 8))
graph = sns.scatterplot(data=df, x="CustomerID", y="recency2")
graph.axhline(100, color="blue")
plt.show();

In [ ]:
fig = px.scatter(df, x="CustomerID", y="recency2")
fig.show()

In [ ]:
fig = px.histogram(df, x="recency2", nbins=70)
fig.show()

# Calculating Frequency

1. Make a copy of dfuk and drop duplicates

In [ ]:
dfUK_copy = df.copy()
dfUK_copy.head(3)

In [ ]:
print("There are", dfUK_copy.duplicated(subset=None, keep='first').sum(), "duplicated observations in the dataset.")
print(dfUK_copy.duplicated(subset=None, keep='first').sum(), "Duplicated observations are dropped!")
dfUK_copy.drop_duplicates(keep='first', inplace=True)

2. Calculate the frequency of purchases

In [ ]:
customer_frequency = dfUK_copy.groupby('CustomerID', as_index=False)['TOTAL_ORDERS'].nunique()
customer_frequency.rename(columns={'TOTAL_ORDERS':'Frequency'}, inplace=True)
customer_frequency.sort_values(by='Frequency', ascending=False)

In [ ]:
customer_frequency.nlargest(5, "Frequency")

3. Plot RFM distributions

In [ ]:
fig = px.scatter(customer_frequency, x="CustomerID", y="Frequency")
fig.show()

In [ ]:
fig = px.histogram(customer_frequency, x="CustomerID", y="Frequency", nbins=3920)
fig.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.distplot(customer_frequency['Frequency'], kde=False, bins=50);

# Calculating Monetary Values

1. Calculate sum total cost by customers and named "Monetary"

In [ ]:
customer_monetary = dfUK_copy.groupby('CustomerID', as_index=False)['REVENUE'].sum()
customer_monetary.rename(columns={'REVENUE':'Monetary'}, inplace=True)
customer_monetary.sort_values(by='Monetary', ascending=False).head()

In [ ]:
customer_monetary.nlargest(5, "Monetary")

2. Plot RFM distributions

In [ ]:
plt.figure(figsize=(14, 7))
sns.scatterplot(data=customer_monetary.Monetary);

In [ ]:
fig = px.scatter(customer_monetary, x="CustomerID", y="Monetary")
fig.show()

In [ ]:
fig = px.histogram(customer_monetary, x="Monetary", nbins=50)
fig.show()

# Creating RFM Table

For creating the RFM Table, we need to merge the recency, frequency and monetary dataframes.

In [ ]:
dfUK_copy.head(3)

In [ ]:
customer_rfm = pd.merge(pd.merge(customer_recency, customer_frequency, on='CustomerID'), customer_monetary, on='CustomerID')
customer_rfm.head()

In [ ]:
customer_rfm.info()

# CUSTOMER SEGMENTATION WITH RFM SCORES

# Calculating RFM Scoring

1. Dividing the customer_rfm into quarters

In [ ]:
customer_rfm.describe()

In [ ]:
quantiles = customer_rfm.quantile(q = [0.25, 0.50, 0.75])
quantiles

# Creating the RFM Segmentation Table

In [ ]:
def recency_scoring(rfm):
    if rfm.Recency <= 24.0:
        recency_score = 4
    elif rfm.Recency <= 57.0:
        recency_score = 3
    elif rfm.Recency <= 149.0:
        recency_score = 2
    else:
        recency_score = 1
    return recency_score

customer_rfm['Recency_Score'] = customer_rfm.apply(recency_scoring, axis=1)
customer_rfm.sample(10)

In [ ]:
fig = px.pie(df, values = customer_rfm['Recency_Score'].value_counts(), 
             names = (customer_rfm["Recency_Score"].value_counts()).index, 
             title = 'Recency Score Distribution')
fig.show()

In [ ]:
def frequency_scoring(rfm):
    if rfm.Frequency >= 10.0:
        frequency_score = 4
    elif rfm.Frequency >= 5.0:
        frequency_score = 3
    elif rfm.Frequency >= 2.0:
        frequency_score = 2
    else:
        frequency_score = 1
    return frequency_score

customer_rfm['Frequency_Score'] = customer_rfm.apply(frequency_scoring, axis=1)
customer_rfm.sample(10)

In [ ]:
fig = px.pie(df, values = customer_rfm['Frequency_Score'].value_counts(), 
             names = (customer_rfm["Frequency_Score"].value_counts()).index, 
             title = 'Frequency Score Distribution')
fig.show()

In [ ]:
def monetary_scoring(rfm):
    if rfm.Monetary >= 1571.0:
        monetary_score = 4
    elif rfm.Monetary >= 645.0:
        monetary_score = 3
    elif rfm.Monetary >= 298.0:
        monetary_score = 2
    else:
        monetary_score = 1
    return monetary_score

customer_rfm['Monetary_Score'] = customer_rfm.apply(monetary_scoring, axis=1)
customer_rfm.sample(10)

In [ ]:
fig = px.pie(df, values = customer_rfm['Monetary_Score'].value_counts(), 
             names = (customer_rfm["Monetary_Score"].value_counts()).index, 
             title = 'Monetary Score Distribution')
fig.show()

In [ ]:
customer_rfm.info()

3. Now that scored each customer, you'll combine the scores for segmentation.

In [ ]:
def rfm_scoring(customer):
    return str(int(customer['Recency_Score'])) + str(int(customer['Frequency_Score'])) + str(int(customer['Monetary_Score']))


customer_rfm['Customer_RFM_Score'] = customer_rfm.apply(rfm_scoring, axis=1)
customer_rfm.sample(8)

In [ ]:
fig = px.histogram(customer_rfm, x = customer_rfm['Customer_RFM_Score'].value_counts().index, 
                   y = customer_rfm['Customer_RFM_Score'].value_counts().values, 
                   title = 'Customer RFM Score Distribution',
                   labels = dict(x = "Customer_RFM_Score", y ="counts"))
fig.show()

4. Define rfm_level function that tags customers by using RFM_Scrores and Create a new variable RFM_Label.

In [ ]:
customer_rfm['RFM_Label'] = customer_rfm['Recency_Score'] + customer_rfm['Frequency_Score'] + customer_rfm['Monetary_Score']

customer_rfm.sample(8)

In [ ]:
customer_rfm.info()

In [ ]:
customer_rfm.groupby(['Customer_RFM_Score']).size().sort_values(ascending=False)[:]

In [ ]:
customer_rfm[customer_rfm['Customer_RFM_Score']=='111'].head(3)

In [ ]:
customer_rfm[customer_rfm['Customer_RFM_Score']=='222'].head(3)

In [ ]:
fig = px.pie(df, values = customer_rfm['RFM_Label'].value_counts(), 
             names = (customer_rfm["RFM_Label"].value_counts()).index, 
             title = 'RFM Label Distribution')
fig.show()

In [ ]:
customer_rfm['RFM_Label'].min()

In [ ]:
customer_rfm['RFM_Label'].max()

In [ ]:
np.sort(customer_rfm['RFM_Label'].unique())

In [ ]:
segments = {'Customer_Segment':[ 
                                'Champions', 
                                'Potential customer', 
                                'Top Recent Customer', 
                                'Recent Customer', 
                                'Top Customer Needed Attention', 
                                'Customer Needed Attention', 
                                'Top Lost Customer', 
                                'Lost Customer'],
            'RFM':[ 
                   '(3)-(1|2|3|4)-(3|4)', 
                   '(3)-(1|2|3|4)-(1|2)', 
                   '(4)-(1|2|3|4)-(3|4)', 
                   '(4)-(1|2|3|4)-(1|2)',
                   '(2|3)-(1|2|3|4)-(3|4)', 
                   '(2|3)-(1|2|3|4)-(1|2)',
                   '(1)-(1|2|3|4)-(3|4)', 
                   '(1)-(1|2|3|4)-(1|2)',]}

pd.DataFrame(segments)

In [ ]:
def categorizer(rfm):
    
    if (rfm[0] in ['3']) & (rfm[1] in ['1', '2', '3', '4']) & (rfm[2] in ['3', '4']):
        rfm = 'Champions'
 
        
    elif (rfm[0] in ['3']) & (rfm[1] in ['1', '2', '3', '4']) & (rfm[2] in ['1', '2']):
        rfm = 'Potential customers'
    
    elif (rfm[0] in ['4']) & (rfm[1] in ['1', '2', '3', '4']) & (rfm[2] in ['3', '4']):
        rfm = 'Top Recent Customer'
    
    elif (rfm[0] in ['4']) & (rfm[1] in ['1', '2', '3', '4']) & (rfm[2] in ['1', '2']):
        rfm = 'Recent Customer'
    
    elif (rfm[0] in ['2', '3']) & (rfm[1] in ['1', '2', '3', '4']) & (rfm[2] in ['3', '4']):
        rfm = 'Top Customer Needed Attention'    
   
    elif (rfm[0] in ['2', '3']) & (rfm[1] in ['1', '2', '3', '4']) & (rfm[2] in ['1', '2']):
        rfm = 'Customer Needed Attention'
    
    elif (rfm[0] in ['1']) & (rfm[1] in ['1', '2', '3', '4']) & (rfm[2] in ['3', '4']):
        rfm = 'Top Lost Customer'
                
    elif (rfm[0] in ['1']) & (rfm[1] in ['1', '2', '3', '4']) & (rfm[2] in ['1', '2']):
        rfm = 'Lost Customer'
    
    return rfm 
 

In [ ]:
customer_rfm['Customer_Category'] = customer_rfm["Customer_RFM_Score"].apply(categorizer)
customer_rfm

In [ ]:
customer_rfm.groupby('Customer_Category').RFM_Label.mean()

In [ ]:
customer_rfm['Customer_Category'].value_counts(dropna=False, normalize=True)*100

In [ ]:
customer_rfm[customer_rfm['Customer_Category'] == "Recent Customer"].sample(8)

In [ ]:
fig = px.histogram(customer_rfm, 
                   x = customer_rfm['Customer_Category'].value_counts().index, 
                   y = customer_rfm['Customer_Category'].value_counts().values, 
                   title = 'Customer Category Distribution',
                   labels = dict(x = "Customer_Category", y ="counts"))
fig.show()

In [ ]:
fig = px.pie(df, 
             values = customer_rfm['Customer_Category'].value_counts(), 
             names = (customer_rfm["Customer_Category"].value_counts()).index, 
             title = 'Customer Category Distribution')
fig.show()

5. Calculate average values for each RFM_Label, and return a size of each segment.

In [ ]:
customer_rfm

In [ ]:
Avg_RFM_Label = customer_rfm.groupby('Customer_Category').RFM_Label.mean()
Size_RFM_Label = customer_rfm['Customer_Category'].value_counts()
df_customer_segmentation = pd.concat([Avg_RFM_Label, Size_RFM_Label], axis=1).rename(columns={'RFM_Label':'Avg_RFM_Label',
                                                                           'Customer_Category':'Size_RFM_Label'})
df_customer_segmentation

In [ ]:
fig = px.histogram(customer_rfm, 
                   x = customer_rfm.groupby('Customer_Category').RFM_Label.mean().sort_values(ascending=False).index, 
                   y = customer_rfm.groupby('Customer_Category').RFM_Label.mean().sort_values(ascending=False).values, 
                   title = 'The Average of RFM Label',
                   labels = dict(x = "Customer Segments (Categories)", y ="RFM Label Mean Values"))
fig.show()

In [ ]:
fig = px.treemap(df_customer_segmentation, 
                 path=[df_customer_segmentation.index], 
                 values='Avg_RFM_Label', 
                 width=950, height=600)

fig.update_layout(title_text='The Average of Each Customer Segments',
                  title_x=0.5, title_font=dict(size=20)
                  )
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [ ]:
fig = px.histogram(customer_rfm, 
                   x = customer_rfm['Customer_Category'].value_counts().index, 
                   y = customer_rfm['Customer_Category'].value_counts().values, 
                   title = 'The Size of RFM Label',
                   labels = dict(x = "Customer Segments (Categories)", y ="RFM Label Mean Values"))
fig.show()

In [ ]:
fig = px.treemap(df_customer_segmentation, 
                 path=[df_customer_segmentation.index], 
                 values='Size_RFM_Label', 
                 width=950, height=600)

fig.update_layout(title_text='Customer Segmentation',
                  title_x=0.5, title_font=dict(size=20)
                  )
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

# Plotting RFM Segments

1. Create your plot and resize it.

In [ ]:
customer_rfm.head(3)

In [ ]:
segmentation = pd.DataFrame(customer_rfm.Customer_Category.value_counts(dropna=False).sort_values(ascending=False))
segmentation.reset_index(inplace=True)
segmentation.rename(columns={'index':'Customer Category', 'Customer_Category':'The Number Of Customer'}, inplace=True)
segmentation

In [ ]:
plt.figure(figsize=(19, 8))

sns.barplot(data=segmentation, x='Customer Category', y='The Number Of Customer', palette='Oranges_r');

In [ ]:
plt.figure(figsize=(14, 7))
sns.lineplot(x=df_customer_segmentation.index, y=df_customer_segmentation.Avg_RFM_Label)
plt.xticks(rotation=30, fontsize=14);

In [ ]:
fig = px.bar(segmentation, x='Customer Category', y='The Number Of Customer')
fig.show()

Using customer segmentation categories found here we can formulate different marketing strategies and approaches for customer engagement for each type of customer.

Note: The author in the article scores 1 as the highest and 4 as the lowest

2. How many customers do we have in each segment?

In [ ]:
segmentation

Segment Visualization with Plotly Matplotlib Squarify

In [ ]:

fig = plt.gcf()
ax = fig.add_subplot()
fig.set_size_inches(20, 10)
sq.plot(sizes=segmentation['The Number Of Customer'], 
                      label=['Lost Customer', 
                            'Customer Needed Attention', 
                            'Top Recent Customer', 
                            'Champion', 
                            'Top Customer Needed Attention', 
                            'Potential customer', 
                            
                            'Recent Customer', 
                            'Top Lost Customer'], 
                            alpha=0.8, 
                            color=["red", "#48BCF5", "#DD6AE1", "blue", "cyan", "magenta", '#B20CB7', "#A4E919"])
plt.title("RFM Segments", fontsize=18, fontweight="bold")
plt.axis('off')
plt.show()

Segment Visualization with Plotly TreeMap

In [ ]:
fig = px.treemap(segmentation,
                 path=[segmentation['Customer Category']], 
                 values='The Number Of Customer', 
                 width=900, 
                 height=600)
fig.update_layout(title="RFM Segments",
                  title_x = 0.5, title_font = dict(size=18),
                 )
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

# APPLYING K-MEANS CLUSTERING

 Data Pre-Processing & Exploring

In [ ]:
customer_rfm.sample(5)

In [ ]:
customer_rfm.groupby('RFM_Label')[['Recency_Score', 'Frequency_Score', 'Monetary_Score']].describe().style.background_gradient(cmap='RdPu')

 Defining and Plot Feature Correlations

In [ ]:
customer_rfm.corr()

Let's check if there exits a multicollinearity problem among the features.

In [ ]:
df_temp = customer_rfm.corr()

feature =[]
collinear=[]

for col in df_temp.columns:
    for i in df_temp.index:
        if (df_temp[col][i]> .85 and df_temp[col][i] < 1) or (df_temp[col][i]< -.85 and df_temp[col][i] > -1) :
                feature.append(col)
                collinear.append(i)
                print(Fore.RED + f"\033[1mmulticolinearity alert\033[0m between {col} - {i}")
        else:
            print(f"For {col} and {i}, there is \033[1mNO multicollinearity problem\033[0m") 

unique_list = list(set(feature+collinear))

print(colored('*'*80, 'cyan', attrs=['bold']))
print("\033[1mThe total number of strong corelated features:\033[0m", len(unique_list)) 

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(14, 10))

# Getting the Upper Triangle of the co-relation matrix
matrix = np.triu(customer_rfm.corr())

# using the upper triangle matrix as mask 
sns.heatmap(customer_rfm.corr(), annot=True, cmap = sns.cubehelix_palette(8), mask=matrix)

plt.xticks(rotation=45);

In [ ]:
customer_rfm.head(3)

In [ ]:
g = sns.scatterplot(data=customer_rfm, x="RFM_Label", y="Recency_Score", hue="Customer_Category")
g.legend(loc='center left', bbox_to_anchor=(1, 0.85), ncol=1);

In [ ]:
g = sns.scatterplot(data=customer_rfm, x="RFM_Label", y="Monetary_Score", hue="Customer_Category")
g.legend(loc='center left', bbox_to_anchor=(1, 0.85), ncol=1);

In [ ]:
g = sns.scatterplot(data=customer_rfm, x="RFM_Label", y="Frequency_Score", hue="Customer_Category")
g.legend(loc='center left', bbox_to_anchor=(1, 0.85), ncol=1);

It seems that there have been strong correlations between RFM_Label and the features including RFM scores; however, it should be remembered that RFM_Label were derived from these related RFM scores. So the strong relationship among them is inevitable and acceptable and does NOT create any problem for our analysis.

# Visualizing Feature Distributions

In [ ]:
plt.figure(figsize=(20, 20))
sns.pairplot(customer_rfm[['Recency', 'Frequency', 'Monetary','Customer_Category']], hue='Customer_Category');

In [ ]:
plt.figure(figsize=(20, 20))
sns.pairplot(customer_rfm[['Recency', 'Frequency', 'Monetary','Customer_Category']], hue='Customer_Category');

In [ ]:
customer_rfm[['Recency', 'Frequency', 'Monetary']].iplot(kind='histogram', subplots=True, bins=50)

In [ ]:
# plot the distribution of RFM values

f, ax = plt.subplots(figsize=(10, 12))
plt.subplot(3, 1, 1); sns.distplot(customer_rfm.Recency, label = 'Recency')
plt.subplot(3, 1, 2); sns.distplot(customer_rfm.Frequency, label = 'Frequency')
plt.subplot(3, 1, 3); sns.distplot(customer_rfm.Monetary, label = 'Monetary')
plt.style.use('fivethirtyeight')
plt.tight_layout()
plt.show()

# Data Normalization

In [ ]:
customer_rfm[['Recency', 'Frequency', 'Monetary']].sample(10)

In [ ]:
matrix = np.triu(customer_rfm[['Recency','Frequency','Monetary']].corr())
fig, ax = plt.subplots(figsize=(11, 6)) 
sns.heatmap (customer_rfm[['Recency','Frequency','Monetary']].corr(), 
             annot=True, 
             fmt= '.2f', 
             vmin=-1, 
             vmax=1, 
             center=0, 
             cmap='coolwarm',
             mask=matrix, 
             ax=ax);

In [ ]:
customer_rfm.set_index("CustomerID", inplace=True)
cell=customer_rfm.copy()

In [ ]:
# **Handling with Skewness - np.log**

skew_limit = 0.75 # This is our threshold-limit to evaluate skewness. Overall below abs(1) seems acceptable for the linear models. 
skew_vals = customer_rfm[['Recency', 'Frequency', 'Monetary']].skew()
skew_cols = skew_vals[abs(skew_vals)> skew_limit].sort_values(ascending=False)
skew_cols

In [ ]:
rfm_log = customer_rfm[skew_cols.index].copy()

for col in skew_cols.index.values:
    rfm_log[col] = rfm_log[col].apply(np.log1p)

    print(rfm_log.skew())
print()

rfm_log.iplot(kind='histogram', subplots=True, bins=50);

0.75 is our threshold-limit to evaluate skewness. Overall below abs(1) seems acceptable for the linear models.

In [ ]:
#  Interpreting Skewness 

for skew in rfm_log.skew():
    if -0.75 < skew < 0.75:
        print ("A skewness value of", '\033[1m', Fore.GREEN, skew, '\033[0m', "means that the distribution is approx.", '\033[1m', Fore.GREEN, "symmetric", '\033[0m')
    elif  -0.75 < skew < -1.0 or 0.75 < skew < 1.0:
        print ("A skewness value of", '\033[1m', Fore.YELLOW, skew, '\033[0m', "means that the distribution is approx.", '\033[1m', Fore.YELLOW, "moderately skewed", '\033[0m')
    else:
        print ("A skewness value of", '\033[1m', Fore.RED, skew, '\033[0m', "means that the distribution is approx.", '\033[1m', Fore.RED, "highly skewed", '\033[0m')


In [ ]:
rfm_log.head(10)

In [ ]:
rfm_log.iplot(kind='histogram',subplots=True,bins=50);

In [ ]:
# customer_rfm
# rfm_trans = pd.merge(customer_rfm,rfm_log)
# rfm_trans = pd.merge(customer_rfm,rfm_log, on=[])
col=cell.drop(["Recency_Score",'Frequency_Score','Monetary_Score',"Customer_RFM_Score",'RFM_Label','Customer_Category','Monetary'],axis=1)
rfm_trans = pd.concat([col,rfm_log,], axis=1,join='inner')

In [ ]:
rfm_trans.head(5)

In [ ]:
rfm_trans.iplot(kind='histogram', subplots=True, bins=50);

In [ ]:
f, ax = plt.subplots(figsize=(10, 12))
plt.subplot(3, 1, 1); sns.distplot(rfm_trans.Recency, label = 'Recency')
plt.subplot(3, 1, 2); sns.distplot(rfm_trans.Frequency, label = 'Frequency')
plt.subplot(3, 1, 3); sns.distplot(rfm_trans.Monetary, label = 'Monetary')
plt.style.use('default')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (20,20));
sns.pairplot(rfm_trans);

In [ ]:
plt.figure(figsize = (15, 8))
sns.scatterplot(data = rfm_trans);

In [ ]:
plt.figure(figsize = (15, 8))
sns.boxplot(data = rfm_trans);

In [ ]:
rfm_trans.head()

In [ ]:
fig = px.scatter_3d(rfm_trans, 
                    x='Recency',
                    y='Frequency',
                    z='Monetary',
                    color='Frequency')
fig.show();

# K-Means Implementation

Defining the Optimal Number of Clusters

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering
from pyclustertend import hopkins
from sklearn.preprocessing import scale
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import silhouette_samples,silhouette_score
from scipy.cluster.hierarchy import linkage, dendrogram
from yellowbrick.cluster import KElbowVisualizer

In [ ]:

scaler = StandardScaler()
scaler.fit(rfm_trans)

#Store it separately for clustering
rfm_scaled = scaler.transform(rfm_trans)

In [ ]:
hopkins(rfm_scaled, rfm_scaled.shape[0])

Elbow method

In [ ]:
#First : Get the Best KMeans 
ks = range(1, 12)
inertias=[]
for k in ks :
    # Create a KMeans clusters
    kc = KMeans(n_clusters=k, random_state=42)
    kc.fit(rfm_scaled)
    inertias.append(kc.inertia_)

# Plot ks vs inertias
f, ax = plt.subplots(figsize=(15, 8))
plt.plot(ks, inertias, '-o')
plt.xlabel('Number of clusters, k')
plt.ylabel('Inertia')
plt.xticks(ks)
plt.style.use('ggplot')
plt.title('What is the Best Number for KMeans?')
plt.show()

In [ ]:
# we will use rfm_scaled data from mow instead of rfm_log

plt.rcParams['figure.facecolor'] = 'white'
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1, 12))

plt.figure(figsize=(14, 8))
visualizer.fit(rfm_scaled)        # Fit the data to the visualizer
visualizer.show()

In the visualizer above, the KElbowVisualizer fits the model for a range of RFM clusters values from 1 to 11, which is set by the parameter k=(1, 12). When the model is fit with 3 clusters we can see an "elbow" in the graph, which in this case we know to be the optimal number since we created our synthetic dataset with 3 clusters of points.

So it can be concluded that according to YellowBrick Elbow method above we are recommended to apply 3 n_clusters for modelling. In addition we will check Classic Elbow Method for a better decision-making

Silhouette Coefficient

In [ ]:
ssd =[]

K = range(2, 12)

for k in K:
    model = KMeans(n_clusters=k)
    model.fit(rfm_scaled)
    ssd.append(model.inertia_)
    print(f'Silhouette Score for {k} clusters: {silhouette_score(rfm_scaled, model.labels_)}')

In [ ]:
from sklearn.cluster import KMeans

from yellowbrick.cluster import SilhouetteVisualizer

model_3 = KMeans(n_clusters=3, random_state=42)
visualizer = SilhouetteVisualizer(model_3)

visualizer.fit(rfm_scaled)    # Fit the data to the visualizer
visualizer.poof();

The Silhouette Coefficient computes the density of clusters computed by the model. The score is computed by averaging the silhouette coefficient for each sample, computed as the difference between the average intra-cluster distance and the mean nearest-cluster distance for each sample, normalized by the maximum value. This produces a score between 1 and -1, where 1 is highly dense clusters and -1 is completely incorrect clustering.

In SilhouetteVisualizer plots, clusters with higher scores have wider silhouettes, but clusters that are less cohesive will fall short of the average score across all clusters, which is plotted as a vertical dotted red line. In other words, the vertical red-dotted line on the plot indicates the average silhouette score for all observations.

In [ ]:
from yellowbrick.cluster import InterclusterDistance
plt.rcParams["figure.figsize"] = (10, 7)

# Instantiate the clustering model and visualizer
model = KMeans(3)
visualizer = InterclusterDistance(model_3)

visualizer.fit(rfm_scaled)  # Fit the data to the visualizer
visualizer.show()  # Finalize and render the figure

Intercluster distance maps display an embedding of the cluster centers in 2 dimensions with the distance to other centers preserved, e.g. the closer two centers are in the visualization, the closer they are in the original feature space.

Model Fitting

Fit the K-Means Algorithm with the optimal number of clusters you decided and save the model to disk.

WATCH OUT: Be careful when determining the number of "n_clusters" and the expressions corresponding to the "labels" or "decision" columns, as my cluster analysis gives different results each time I run my notebook. Please consider the analysis of your own results as I did.

In [ ]:
# prediction was added

kmeans = KMeans(n_clusters = 3).fit(rfm_scaled)
kmeans.fit_predict(rfm_scaled)
labels = kmeans.labels_
rfm_trans['ClusterID']=labels
rfm_trans

In [ ]:
rfm_trans.ClusterID.value_counts()

In [ ]:
fig = px.pie(df, values = rfm_trans['ClusterID'].value_counts(), 
             names = (rfm_trans['ClusterID'].value_counts()).index, 
             title = 'Predicted Clusters Distribution')
fig.show()

In [ ]:
customer_rfm['ClusterID'] = labels
customer_rfm.sample(20)

 Visualizing the Clusters

1. Create a scatter plot and select cluster centers.

In [ ]:
rfm_trans.sample(8)

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(14, 6)) # sharey=True ile y eksen labels lari ortak kullanirlar.

ax1.set_title('Recency-Frequency')
ax1.set_xlabel('Recency')
ax1.set_ylabel('Frequency')
ax1.scatter(rfm_trans.iloc[:, 0], rfm_trans.iloc[:, 1], c=kmeans.labels_, cmap="rainbow")
ax1.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=300, alpha=0.9, label = 'Centroids')

ax2.set_title("Frequency-Monetary")
ax2.set_xlabel('Frequency')
ax2.set_ylabel('Monetary')
ax2.scatter(rfm_trans.iloc[:, 1], rfm_trans.iloc[:, 2], c=kmeans.labels_,cmap="rainbow")
ax2.scatter(kmeans.cluster_centers_[:, 1], kmeans.cluster_centers_[:, 2], s=300, alpha=0.9, label = 'Centroids')

ax3.set_title("Recency-Monetary")
ax3.set_xlabel('Recency')
ax3.set_ylabel('Monetary')
ax3.scatter(rfm_trans.iloc[:, 0], rfm_trans.iloc[:, 2], c=kmeans.labels_,cmap="rainbow")
ax3.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 2], s=300, alpha=0.9, label = 'Centroids');

sualize Cluster Id vs Recency, Cluster Id vs Frequency and Cluster Id vs Monetary using Box plot. Also evaluate the results.

In [ ]:
rfm_trans.sample(8)

In [ ]:
plt.figure(figsize=(15,6))

plt.subplot(1, 3, 1)
sns.boxplot(rfm_trans['ClusterID'], rfm_trans['Recency'])

plt.subplot(1, 3, 2)
sns.boxplot(rfm_trans['ClusterID'], rfm_trans['Frequency'])

plt.subplot(1, 3, 3)
sns.boxplot(rfm_trans['ClusterID'], rfm_trans['Monetary'])
plt.show()

In [ ]:
for i in rfm_trans.drop("ClusterID", axis=1):
    rfm_trans[i].iplot(kind="box", title=i, boxpoints="all", color='lightseagreen')

 Assigning the Label

In [ ]:
rfm_trans['Labels'] = rfm_trans['ClusterID'].map({0:'Lost/Needed Attention Customers', 1:'Current Custemer', 2:'Top/Best Customer'})
rfm_trans.sample(8) 

In [ ]:
rfm_trans[rfm_trans['Labels'] == "Lost/Needed Attention Customers"]

In [ ]:
rfm_trans[rfm_trans['Labels'] == "Top/Best Customer"]


In [ ]:
rfm_trans[rfm_trans['Labels'] == "Current Custemer"]

In [ ]:
customer_rfm.sample(10)

In [ ]:
customer_rfm[customer_rfm['ClusterID'] == 0].sample(10)

In [ ]:
customer_rfm[customer_rfm['ClusterID'] == 1].sample(10)

In [ ]:
customer_rfm[customer_rfm['ClusterID'] == 2].sample(10)

 Conclusion


- Cluster 0 : The first cluster is more related to the "Lost/Needed Attention Customers" who used to visit and shopping at different frequencies or who are still customer but haven’t been visiting recently. The store/department should bring them back with relevant personalized promotions, and run surveys to find out what went wrong and avoid losing them to a competitor.

- Cluster 1 : The second cluster belongs to the "Current Custemer" who have been shopping at different frequencies and spending in varying amounts. The store/department should offer membership or loyalty programs or recommend related products to upsell them and help them become its Loyalists or Champions, and offer renewals and helpful products to encourage another purchase.

- Cluster 2 : The third cluster can be interpreted as "Top/Best Customers" who have been shopping in average frequency or most recently, but spent a good amount. The store/department should reward or offer membership or loyalty programs them to become its Loyalists or Champions.

In [ ]:
rfm_trans.shape

In [ ]:
customer_rfm.shape

In [ ]:
# column to added from first dataframe to second
extracted_col = rfm_trans['Labels']

# Second dataframe after adding column from first dataframe
customer_rfm.insert(10, "K-Means Predicted Cluster", extracted_col)
# customer_rfm = customer_rfm.join(extracted_col)
customer_rfm['K-Means Predicted Cluster'] = rfm_trans['Labels'].values
customer_rfm.sample(8)

In [ ]:
customer_rfm['Customer_Category'].value_counts()

In [ ]:
RFM_Customer_Segment = pd.DataFrame(customer_rfm['Customer_Category'].value_counts(dropna=False).sort_values(ascending=False))
RFM_Customer_Segment.reset_index(inplace=True)
RFM_Customer_Segment.rename(columns={'index':'RFM Customer Segment', 'Customer_Category':'The Number Of Customer'}, inplace=True)
RFM_Customer_Segment

In [ ]:
customer_rfm['K-Means Predicted Cluster'].value_counts()

In [ ]:
KMeans_Predicted_Clusters = pd.DataFrame(customer_rfm['K-Means Predicted Cluster'].value_counts(dropna=False).sort_values(ascending=False))
KMeans_Predicted_Clusters.reset_index(inplace=True)
KMeans_Predicted_Clusters.rename(columns={'index':'K-Means Predicted Cluster', 'K-Means Predicted Cluster':'The Number Of Customer'}, inplace=True)
KMeans_Predicted_Clusters

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=customer_rfm['K-Means Predicted Cluster'].values,
    y=customer_rfm['K-Means Predicted Cluster'].index,
    name='Predicted-Clusters By K-Means',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=customer_rfm['Customer_Category'].values,
    y=customer_rfm['Customer_Category'].index,
    name='Customer-Segment By RFM Analysis',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='relative', 
                  xaxis_tickangle=-45, 
                  title="The Comparison of Predicted-Clusters By K-Means vs Customer-Segment By RFM Analysis")
fig.update_traces(dict(marker_line_width=0))
fig.show()